In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os
from time import sleep

email =input("Email")
passaword = input("Senha")
filial = input("Filial")
driver = webdriver.Chrome()
driver.maximize_window()
def login(driver):
    driver.get('https://one.bees.com/order-management')
    wait = WebDriverWait(driver, 30)
    field_email = wait.until(EC.element_to_be_clickable(("id", "signInName")))
    field_email.send_keys(email)
    button_next = driver.find_element("id","next")
    button_next.click()
    field_password = wait.until(EC.element_to_be_clickable(('id','password')))
    field_password.send_keys(passaword)
    button_next = wait.until(EC.element_to_be_clickable(("id","next"))).click()
    #terms_button = wait.until(EC.element_to_be_clickable(( 'id', 'terms'))).click()
    #policies_button = wait.until(EC.element_to_be_clickable(('id','policies'))).click()
    #continue_button = wait.until(EC.element_to_be_clickable(('xpath','//*[@id="radix-0"]/div[3]/div/button[2]'))).click()
    understand_button = wait.until(EC.element_to_be_clickable(('xpath','//*[@id="single-spa-application:@admin-portal/main-mfe"]/section/header/div[1]/div/div[1]/div[3]/div[2]/button'))).click()
    
login(driver)


In [42]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
wait = WebDriverWait(driver, 10)
soup = BeautifulSoup(driver.page_source, 'html.parser')
pedidos = soup.findAll("tr")
lista_pedidos = []

for pedido in pedidos:
    dados = pedido.findAll('td')
    if len(dados) < 5:
        continue

    numero_pedido = dados[0].get_text().strip()
    data_pedido = dados[1].get_text().strip()
    data_entrega = dados[2].get_text().strip()
    responsavel = dados[3].get_text().strip()
    valor = dados[4].get_text().strip()

    lista_pedidos.append({
        "Numero Pedido": numero_pedido,
        "Data Pedido": data_pedido,
        "Data Entrega": data_entrega,
        "Responsavel": responsavel,
        "Valor": valor
    })

lista_numero = [pedido["Numero Pedido"] for pedido in lista_pedidos]

detalhes = []
for item in lista_numero:
    link = f"https://one.bees.com/order-management/active-orders/{item}"
    driver.get(link)
    
    wait.until(EC.presence_of_element_located((By.XPATH, '//tbody[@role="rowgroup"]')))
    
    time.sleep(5) 
    try:
        vermais_button = driver.find_elements(By.XPATH, '//*[@id="single-spa-application:@supplier-portal/order-management-beta-mfe"]/div/div[8]/div[3]/div[1]/div[4]/a')
        if vermais_button:
            vermais_button[0].click() 
    except Exception as e:
        print(f"Erro ao tentar clicar no 'Ver Mais' para o pedido {item}: {e}")
        continue

    lista_produtos = []
    try:
        wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="single-spa-application:@supplier-portal/order-management-beta-mfe"]/div/div[9]/div[3]/div/div/table/tbody')))
        tbody = driver.find_element(By.XPATH, '//*[@id="single-spa-application:@supplier-portal/order-management-beta-mfe"]/div/div[9]/div[3]/div/div/table/tbody')
        produtos = tbody.find_elements(By.XPATH, './/tr[@role="row"]')

        for produto in produtos:
            nome_sku = produto.find_element(By.XPATH, './/td[1]//div/div[2]/p[1]').text.strip()
            sku = produto.find_element(By.XPATH, './/td[1]//div/div[2]/p[2]').text.strip()
            wait.until(EC.presence_of_element_located((By.XPATH, './/td[1]//div/div[2]/div/p')))
            preco = produto.find_element(By.XPATH, './/td[1]//div/div[2]/div/p').text.strip()
            wait.until(EC.presence_of_element_located((By.XPATH, './/td[1]//div/div[2]/div/p')))
            quantidade_pedida = produto.find_element(By.XPATH, './/td[2]').text.strip()
            quantidade_preparar = produto.find_element(By.XPATH, './/td[3]').text.strip()

            _numero_pedido = driver.find_element(By.XPATH, '//*[@id="single-spa-application:@supplier-portal/order-management-beta-mfe"]/div/div[2]/div[1]/h2').text.strip()
            _numero_pedido = _numero_pedido.replace('Pedido #', '')
            cd=driver.find_element('xpath','//*[@id="single-spa-application:@supplier-portal/order-management-beta-mfe"]/div/div[4]/div[2]/p[2]').text.strip()
            documento = driver.find_element(By.XPATH, '//*[@id="single-spa-application:@supplier-portal/order-management-beta-mfe"]/div/div[8]/div[3]/div[1]/div[2]/p[2]').text.strip()
            try:
                ie_element = driver.find_elements(By.XPATH, '//*[@id="single-spa-application:@supplier-portal/order-management-beta-mfe"]/div/div[8]/div[3]/div[1]/div[2]/p[3]')
                ie = ie_element[0].text.strip() if ie_element else None
            except Exception as e:
                ie_element = "ISENTO"
                if ie_element == None:"ISENTO"
            nome_comercial = driver.find_element(By.XPATH, '//*[@id="single-spa-application:@supplier-portal/order-management-beta-mfe"]/div/div[8]/div[3]/div[1]/div[3]/p[2]').text.strip()
            cidade_uf = driver.find_element(By.XPATH, '//*[@id="single-spa-application:@supplier-portal/order-management-beta-mfe"]/div/div[8]/div[3]/div[1]/div[5]/p[3]').text.strip()
            endereco = driver.find_element(By.XPATH, '//*[@id="single-spa-application:@supplier-portal/order-management-beta-mfe"]/div/div[8]/div[3]/div[1]/div[5]/p[2]').text.strip()
            cep = driver.find_element(By.XPATH, '//*[@id="single-spa-application:@supplier-portal/order-management-beta-mfe"]/div/div[8]/div[3]/div[1]/div[5]/p[4]').text.strip()
            _data_pedido = driver.find_element(By.XPATH, '//*[@id="single-spa-application:@supplier-portal/order-management-beta-mfe"]/div/div[8]/div[3]/div[2]/div[1]/p[2]').text.strip()
            forma_pgto = driver.find_element(By.XPATH, '//*[@id="single-spa-application:@supplier-portal/order-management-beta-mfe"]/div/div[8]/div[3]/div[2]/div[2]/p[2]').text.strip()
            status = driver.find_element(By.XPATH, '//*[@id="single-spa-application:@supplier-portal/order-management-beta-mfe"]/div/div[4]/div[1]/p[2]').text.strip()
            try:
                contato1_element = driver.find_elements(By.XPATH, '//*[@id="single-spa-application:@supplier-portal/order-management-beta-mfe"]/div/div[8]/div[3]/div[1]/div[4]/p[2]')
                contato1 = contato1_element[0].text.strip() if contato1_element else "teste@teste.com"
            except Exception as e:
                contato1 = "teste@teste.com"
                print(f"Erro ao coletar contato2 para o pedido {item}: {e}")
            try:

                contato2_element = driver.find_elements(By.XPATH, '//*[@id="single-spa-application:@supplier-portal/order-management-beta-mfe"]/div/div[8]/div[3]/div[1]/div[4]/p[3]')
                contato2 = contato2_element[0].text.strip() if contato2_element else None
            except Exception as e:
                contato2 = None
                print(f"Erro ao coletar contato2 para o pedido {item}: {e}")
            try:

                contato3_element = driver.find_elements(By.XPATH, '//*[@id="single-spa-application:@supplier-portal/order-management-beta-mfe"]/div/div[8]/div[3]/div[1]/div[4]/p[4]')
                contato3 = contato3_element[0].text.strip() if contato3_element else None
            except Exception as e:
                contato3 = None
                print(f"Erro ao coletar contato2 para o pedido {item}: {e}")
            detalhes.append({
                "Documento": documento,
                "IE":ie,
                "Forma de Pagamento": forma_pgto,
                "SKU": sku,
                "Preço": preco,
                "Quantidade Pedida": quantidade_pedida,
                "Nome Comercial": nome_comercial,
                "Numero Pedido": _numero_pedido,
                "Contato 1": contato1,
                "Contato 2": contato2,
                "Contato 3": contato3,
                "Centro de Distribuição":cd,
                "Status": status,
                "Nome do produto": nome_sku,
                "Quantidade Preparar": quantidade_preparar,
                "CEP":cep,
                "Endereço de Entrega": endereco,
                "Cidade/UF":cidade_uf,
                "Data Pedido": _data_pedido,
                "Data Entrega": data_entrega
            })
    except Exception as e:
        print(f"Erro ao coletar detalhes do pedido {item}: {e}")

df = pd.DataFrame(detalhes)

In [43]:
import pandas as pd

df = pd.DataFrame(detalhes)
df2 = pd.DataFrame(lista_pedidos)
df['Numero Pedido'] = df['Numero Pedido'].astype(str)  
df2['Numero Pedido'] = df2['Numero Pedido'].astype(str)

tabela_pedidos = pd.merge(df, df2, on="Numero Pedido")

tabela_pedidos = tabela_pedidos.drop(columns=['Data Pedido_y'])
tabela_pedidos = tabela_pedidos.rename(columns={'Data Pedido_x': 'Data Pedido', 'Data Entrega_x': 'Data Entrega'})

tabela_pedidos['Contato 1'] = tabela_pedidos['Contato 1'].str.replace(r"[()\s-]", "", regex=True)
tabela_pedidos['Contato 2'] = tabela_pedidos['Contato 2'].str.replace(r"[()\s-]", "", regex=True)

caminho = r'G:\Drives compartilhados\Cadastro BEES'
pedidos = tabela_pedidos.to_csv(f'{caminho}/Pedidos_A_Preparar_{filial}.csv', index=False, encoding='utf-8-sig')

In [33]:
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
wait = WebDriverWait(driver, 30)

cidades_interior = ['ITATIAIA', 'RESENDE', 'PORTO REAL', 'QUATIS', 'BARRA MANSA', 'VOLTA REDONDA', 'ARROZAL', 'PINHEIRAL', 'PIRAI', 'BARRA DO PIRAI', 'MENDES', 'VASSOURA', 'RIO CLARO']

driver.get("https://web.whatsapp.com/")
while len(driver.find_elements(By.ID, "side")) < 1:
    time.sleep(1)
time.sleep(5)
driver.find_element('xpath','//span[@data-icon="settings-outline"]').click()
time.sleep(3)
driver.find_element('xpath','//div[text()="Conversas"]').click()
time.sleep(3)
driver.find_element('xpath','//*[@id="app"]/div/div[3]/div/div[2]/div[1]/span/div/span/div/div/div/div/div/div[2]/div[6]/div/div').click()
time.sleep(3)
driver.find_element('xpath','//*[@id="app"]/div/div[3]/div/header/div/div/div/div/span/div/div[1]/div[1]/div/div/span').click()
time.sleep(3)

def enviar_mensagem_pedido(driver, grupo, pedido):

    estabelecimento = grupo['Nome Comercial'].iloc[0]
    contato1 = grupo['Contato 1'].iloc[0]
    contato2 = grupo['Contato 2'].iloc[0]
    contato3 = grupo['Contato 3'].iloc[0]
    forma_pgto = grupo['Forma de Pagamento'].iloc[0]
    endereco = grupo['Endereço de Entrega'].iloc[0]
    cidade_uf = grupo['Cidade/UF'].iloc[0]
    cep = grupo['CEP'].iloc[0]
    documento = grupo['Documento'].iloc[0]
    ie = grupo['IE'].iloc[0]
    data_entrega = grupo['Data Entrega'].iloc[0]
    responsavel = grupo['Responsavel'].iloc[0]


    mensagem = f"Novo Pedido *{pedido}*\n*Responsavel*: {responsavel}\n*Estabelecimento*: {estabelecimento}\n*Contato 1*: {contato1}\n*Contato 2*: {contato2}\n*Contato 3*: {contato3}\n*Forma de Pagamento*: {forma_pgto}\n*Endereço de Entrega*: {endereco}\n*Cidade/UF*: {cidade_uf}\n*CEP*: {cep}\n{documento}\n{ie}\n*Data Entrega*: {data_entrega}\n"
    
    for _, row in grupo.iterrows():
        mensagem += f"*Produto*: {row['Nome do produto']} | Quantidade: {row['Quantidade Pedida']} | Preço: R{row['Preço']}\n"

    return mensagem 

def verificar_condicoes_e_enviar_mensagem(driver, tabela_pedidos):
    for pedido, grupo in tabela_pedidos.groupby('Numero Pedido'):
        cidade_uf = grupo['Cidade/UF'].iloc[0]
        documento = grupo['Documento'].iloc[0]

  
        mensagem = enviar_mensagem_pedido(driver, grupo, pedido)

        if not any(cidade in cidade_uf for cidade in cidades_interior) and "CPF" in documento and "RJ" in cidade_uf and "MIGUEL PEREIRA" not in cidade_uf:
          
            field_find = wait.until(EC.presence_of_element_located(('xpath','(//div[@role="textbox"])[1]')))
            field_find.send_keys("NF PEDIDOS VENDAS bebida in box Bees")
            field_find.send_keys(Keys.ENTER)
            field_message = wait.until(EC.presence_of_element_located(('xpath','(//div[@role="textbox"])[2]')))
            field_message.send_keys(mensagem)
            botao_enviar = driver.find_element(By.XPATH, "//button[@aria-label='Enviar']")
            botao_enviar.click()
            
       
        elif any(cidade in cidade_uf for cidade in cidades_interior):
            
            field_find = wait.until(EC.presence_of_element_located(('xpath','(//div[@role="textbox"])[1]')))
            field_find.send_keys("Pedidos BEES - RJ Interior")
            field_find.send_keys(Keys.ENTER)
            field_message = wait.until(EC.presence_of_element_located(('xpath','(//div[@role="textbox"])[2]')))
            field_message.send_keys(mensagem)
            botao_enviar = driver.find_element(By.XPATH, "//button[@aria-label='Enviar']")
            botao_enviar.click()


verificar_condicoes_e_enviar_mensagem(driver, tabela_pedidos)

In [ ]:
import pandas
caminho = r"G:\Drives compartilhados\Cadastro BEES\Cadastro.xlsx"
arquivo = pd.read_excel(caminho,sheet_name="MG")
display(arquivo)

In [ ]:
import smtplib
import pandas

caminho = r"G:\Drives compartilhados\Cadastro BEES\Cadastro.xlsx"
arquivo = pd.read_excel(caminho,sheet_name="MG")
display(arquivo)

# Função para enviar e-mails
def enviar_email(responsavel, estabelecimento, contato1, contato2, contato3, endereco, cidade_uf, cep, documento, ie):  
    mensagem = f"""
    <p><b>Responsável:</b> {responsavel}</p>
    <p><b>Estabelecimento:</b> {estabelecimento}</p>
    <p>{documento}</p>
    <p>{ie}</p>
    <p><b>Contato 1:</b> {contato1}</p>
    <p><b>Contato 2:</b> {contato2}</p>
    <p><b>Contato 3:</b> {contato3}</p>
    <p><b>Endereço de Entrega:</b> {endereco}</p>
    <p><b>Cidade/UF:</b> {cidade_uf}</p>
    <p><b>CEP:</b> {cep}</p>
    """
 
    msg = email.message.Message()
    msg['Subject'] = "CADASTRO BEES"
    msg['From'] = 'leonardo.campos@rigarr.com.br'
    msg['To'] = 'leonardo.campos@rigarr.com.br'
    password = 'vfjwgbxpogleobok'  # Cuidado para não expor a senha em código fonte
    msg.add_header('Content-Type', 'text/html')
    msg.set_payload(mensagem)

    # Configuração do servidor SMTP
    try:
        s = smtplib.SMTP('smtp.gmail.com:587')
        s.starttls()
        s.login(msg['From'], password)
        s.sendmail(msg['From'], [msg['To']], msg.as_string().encode('utf-8'))
        print(f"E-mail enviado com sucesso para {responsavel} (Documento: {documento}).")
    except Exception as e:
        print(f"Erro ao enviar e-mail para {responsavel} (Documento: {documento}): {e}")
    finally:
        s.quit()

for index,row in arquivo.iterrows():
    enviar_email(
        documento=row["Documento"],
        ie=row['IE'],
        responsavel=row['Responsavel'],
        estabelecimento=row['Nome Comercial'],
        contato1=row['Email'],
        contato2=['telefone '],
        contato3=['Telefone '],
        endereco=['Endereço'],
        cidade_uf=['Cidade'],
        cep=['CEP']
    )


In [7]:
import pandas as pd
import re
import urllib.parse
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import os

# Função para limpar e formatar números de telefone
def clean_phone_numbers(row, columns):
    """Extrai, limpa e formata números de telefone de múltiplas colunas."""
    cleaned_numbers = set()  # Usar um set para evitar números duplicados
    for column in columns:
        if pd.isna(row[column]):
            continue
        phone_numbers = str(row[column]).split(',')  # Divide por vírgula
        for number in phone_numbers:
            number = re.sub(r'[^\d]', '', number)  # Remove caracteres especiais
            if number:  # Adiciona se não for vazio
                cleaned_numbers.add(str(number))  # Garantir como string
    return list(cleaned_numbers)


# Carregar os dados da planilha principal
tabela_pedidos = pd.read_csv(f'G:\Drives compartilhados\Cadastro BEES\Pedidos_A_Preparar_Rigarr.csv')


# Configurar o driver do WhatsApp Web
driver.get("https://web.whatsapp.com/")
print("Aguardando conexão com o WhatsApp Web...")
while len(driver.find_elements(By.ID, "side")) < 1:
    time.sleep(1)
print("Conectado ao WhatsApp Web.")

# Conjunto de telefones processados para evitar envios duplicados na execução atual
telefones_processados = set()

# Enviar mensagens para pedidos pendentes
for index, row in tabela_pedidos.iterrows():
    if row['Status'] == 'Enviado':
        continue

    numero_pedido = row['Numero Pedido']
    phone = row['Telefone']

    # Verificar se o telefone já foi processado nesta execução
    if phone in telefones_processados:
        print(f"Número {phone} já processado nesta execução. Ignorando.")
        continue

    # Buscar os dados do pedido na tabela principal
    pedido_data = tabela_pedidos[tabela_pedidos['Numero Pedido'] == numero_pedido]
    endereco = pedido_data.iloc[0]['Endereço de Entrega']
    cep = pedido_data.iloc[0]['CEP']
    cidade_uf = pedido_data.iloc[0]['Cidade/UF']
    forma_pagamento = pedido_data.iloc[0]['Forma de Pagamento']

    # Criar lista de produtos
    produtos = ""
    for _, produto_row in pedido_data.iterrows():
        nome_produto = produto_row['Nome do produto']
        preco = produto_row['Preço']
        quantidade = produto_row['Quantidade Pedida']
        produtos += f"- {nome_produto} (Quantidade: {quantidade}, Preço: {preco})\n"

    # Construir a mensagem
    message = (f"Olá! Recebemos seu pedido de bebidas, nº *{numero_pedido}* no BEES-Ambev 🐝.\n"
               f"Produtos no pedido:\n"
               f"{produtos}\n"
               f"Endereço de entrega:\n"
               f"{endereco},\n{cidade_uf},\n*CEP*: {cep}.\n"
               f"Forma de pagamento: {forma_pagamento}.*Não aceitamos forma de pagamento diferente da nota.*\n"
               f"Entrega de Segunda a Sábado, entre 8h e 18h.\n"
               f"Caso o endereço esteja incorreto, por favor abra uma solicitação de correção no app BEES.\n")

    # Enviar mensagem no WhatsApp
    texto = urllib.parse.quote(message)
   
    link = f"https://web.whatsapp.com/send?phone={phone}&text={texto}"
    driver.get(link)
    while len(driver.find_elements(By.ID, "side")) < 1:
        time.sleep(1)  
    try:
        time.sleep(5)
        driver.find_element(By.XPATH, '//button[@aria-label="Enviar"]').send_keys(Keys.ENTER)
        time.sleep(2)
        print(f"Mensagem enviada para {phone} - Pedido {numero_pedido}")
        tabela_pedidos.loc[index, 'Status'] = 'Enviado'
        telefones_processados.add(phone)  # Adicionar telefone ao conjunto processado
    except Exception as e:
        print(f"Erro ao enviar mensagem para {phone} - Pedido {numero_pedido}")


<>:26: SyntaxWarning: invalid escape sequence '\D'
<>:26: SyntaxWarning: invalid escape sequence '\D'
C:\Users\leonardo.campos\AppData\Local\Temp\ipykernel_19460\3049083061.py:26: SyntaxWarning: invalid escape sequence '\D'
  tabela_pedidos = pd.read_csv(f'G:\Drives compartilhados\Cadastro BEES\Pedidos_A_Preparar_Rigarr.csv')


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=131.0.6778.140)
Stacktrace:
	GetHandleVerifier [0x00007FF640126CC5+28821]
	(No symbol) [0x00007FF640093850]
	(No symbol) [0x00007FF63FF3578A]
	(No symbol) [0x00007FF63FF0F4F5]
	(No symbol) [0x00007FF63FFB6247]
	(No symbol) [0x00007FF63FFCECE2]
	(No symbol) [0x00007FF63FFAF0A3]
	(No symbol) [0x00007FF63FF7A778]
	(No symbol) [0x00007FF63FF7B8E1]
	GetHandleVerifier [0x00007FF64045FCCD+3408029]
	GetHandleVerifier [0x00007FF64047743F+3504143]
	GetHandleVerifier [0x00007FF64046B61D+3455469]
	GetHandleVerifier [0x00007FF6401EBDCB+835995]
	(No symbol) [0x00007FF64009EB6F]
	(No symbol) [0x00007FF64009A824]
	(No symbol) [0x00007FF64009A9BD]
	(No symbol) [0x00007FF64008A1A9]
	BaseThreadInitThunk [0x00007FFDB1E6E8D7+23]
	RtlUserThreadStart [0x00007FFDB349FBCC+44]
